In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

# Problem 1

In [5]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  tf_reg=tf.placeholder(tf.float32)
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+tf_reg*tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [20]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,tf_reg:2e-3}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 24.356722
Minibatch accuracy: 8.6%
Validation accuracy: 7.1%
Minibatch loss at step 500: 2.930099
Minibatch accuracy: 72.7%
Validation accuracy: 76.1%
Minibatch loss at step 1000: 1.254577
Minibatch accuracy: 80.5%
Validation accuracy: 80.2%
Minibatch loss at step 1500: 0.816564
Minibatch accuracy: 82.8%
Validation accuracy: 80.2%
Minibatch loss at step 2000: 0.836987
Minibatch accuracy: 79.7%
Validation accuracy: 82.0%
Minibatch loss at step 2500: 0.600330
Minibatch accuracy: 85.2%
Validation accuracy: 81.5%
Minibatch loss at step 3000: 0.913138
Minibatch accuracy: 75.8%
Validation accuracy: 82.0%
Test accuracy: 89.0%


In [21]:
batch_size = 128
nodes=1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  tf_reg=tf.placeholder(tf.float32)
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size,nodes]))
  biases_1 = tf.Variable(tf.zeros([nodes]))
  weights_2 = tf.Variable(
    tf.truncated_normal([nodes,num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  layer_1_training=tf.nn.relu(tf.matmul(tf_train_dataset,weights_1)+biases_1)
  logits = tf.matmul(layer_1_training, weights_2) + biases_2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+tf_reg*(tf.nn.l2_loss(weights_1)+tf.nn.l2_loss(weights_2))
  
  # Optimizer.
  
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  layer_1_validation=tf.nn.relu(tf.matmul(tf_valid_dataset,weights_1)+biases_1)
  layer_1_test=tf.nn.relu(tf.matmul(tf_test_dataset,weights_1)+biases_1)
  valid_prediction = tf.nn.softmax(
    tf.matmul(layer_1_validation, weights_2) + biases_2)
  test_prediction = tf.nn.softmax(tf.matmul(layer_1_test, weights_2) + biases_2)

In [22]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,tf_reg:2e-3}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 970.008118
Minibatch accuracy: 3.9%
Validation accuracy: 28.0%
Minibatch loss at step 500: 233.302582
Minibatch accuracy: 76.6%
Validation accuracy: 80.8%
Minibatch loss at step 1000: 83.134079
Minibatch accuracy: 83.6%
Validation accuracy: 83.7%
Minibatch loss at step 1500: 30.687077
Minibatch accuracy: 88.3%
Validation accuracy: 85.2%
Minibatch loss at step 2000: 11.714674
Minibatch accuracy: 85.2%
Validation accuracy: 86.6%
Minibatch loss at step 2500: 4.491023
Minibatch accuracy: 93.0%
Validation accuracy: 86.2%
Minibatch loss at step 3000: 2.169470
Minibatch accuracy: 84.4%
Validation accuracy: 86.8%
Test accuracy: 92.9%


# Problem 2

In [23]:
batch_size = 128
nodes=1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  tf_reg=tf.placeholder(tf.float32)
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size,nodes]))
  biases_1 = tf.Variable(tf.zeros([nodes]))
  weights_2 = tf.Variable(
    tf.truncated_normal([nodes,num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  layer_1_training=tf.nn.relu(tf.matmul(tf_train_dataset,weights_1)+biases_1)
  logits = tf.matmul(layer_1_training, weights_2) + biases_2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+tf_reg*(tf.nn.l2_loss(weights_1)+tf.nn.l2_loss(weights_2))
  
  # Optimizer.
  
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  layer_1_validation=tf.nn.relu(tf.matmul(tf_valid_dataset,weights_1)+biases_1)
  layer_1_test=tf.nn.relu(tf.matmul(tf_test_dataset,weights_1)+biases_1)
  valid_prediction = tf.nn.softmax(
    tf.matmul(layer_1_validation, weights_2) + biases_2)
  test_prediction = tf.nn.softmax(tf.matmul(layer_1_test, weights_2) + biases_2)

In [24]:
num_steps = 3001
batches=100
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = step%batches
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,tf_reg:2e-3}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1009.813965
Minibatch accuracy: 11.7%
Validation accuracy: 34.5%
Minibatch loss at step 100: 517.090637
Minibatch accuracy: 98.4%
Validation accuracy: 70.5%
Minibatch loss at step 200: 422.655884
Minibatch accuracy: 98.4%
Validation accuracy: 70.1%
Minibatch loss at step 300: 345.669159
Minibatch accuracy: 100.0%
Validation accuracy: 69.5%
Minibatch loss at step 400: 282.981934
Minibatch accuracy: 100.0%
Validation accuracy: 69.5%
Minibatch loss at step 500: 231.662735
Minibatch accuracy: 100.0%
Validation accuracy: 69.5%
Minibatch loss at step 600: 189.650253
Minibatch accuracy: 100.0%
Validation accuracy: 69.5%
Minibatch loss at step 700: 155.257065
Minibatch accuracy: 100.0%
Validation accuracy: 69.5%
Minibatch loss at step 800: 127.101082
Minibatch accuracy: 100.0%
Validation accuracy: 69.6%
Minibatch loss at step 900: 104.051003
Minibatch accuracy: 100.0%
Validation accuracy: 69.5%
Minibatch loss at step 1000: 85.181351
Minibatch accuracy: 100

# Problem 3

In [30]:
batch_size = 128
nodes=1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  tf_reg=tf.placeholder(tf.float32)
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size,nodes]))
  biases_1 = tf.Variable(tf.zeros([nodes]))
  weights_2 = tf.Variable(
    tf.truncated_normal([nodes,num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  layer_1_training=tf.nn.relu(tf.matmul(tf_train_dataset,weights_1)+biases_1)
  drop_out=tf.nn.dropout(layer_1_training,0.5)
  logits = tf.matmul(drop_out, weights_2) + biases_2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+tf_reg*(tf.nn.l2_loss(weights_1)+tf.nn.l2_loss(weights_2))
  
  # Optimizer.
  
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  layer_1_validation=tf.nn.relu(tf.matmul(tf_valid_dataset,weights_1)+biases_1)
  layer_1_test=tf.nn.relu(tf.matmul(tf_test_dataset,weights_1)+biases_1)
  valid_prediction = tf.nn.softmax(
    tf.matmul(layer_1_validation, weights_2) + biases_2)
  test_prediction = tf.nn.softmax(tf.matmul(layer_1_test, weights_2) + biases_2)

In [31]:
num_steps = 3001
batches=100
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = step%batches
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,tf_reg:2e-3}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1120.916016
Minibatch accuracy: 12.5%
Validation accuracy: 26.2%
Minibatch loss at step 100: 522.680664
Minibatch accuracy: 97.7%
Validation accuracy: 73.9%
Minibatch loss at step 200: 425.085754
Minibatch accuracy: 98.4%
Validation accuracy: 74.5%
Minibatch loss at step 300: 347.258942
Minibatch accuracy: 100.0%
Validation accuracy: 74.8%
Minibatch loss at step 400: 284.310577
Minibatch accuracy: 100.0%
Validation accuracy: 74.8%
Minibatch loss at step 500: 232.778915
Minibatch accuracy: 100.0%
Validation accuracy: 75.0%
Minibatch loss at step 600: 190.651047
Minibatch accuracy: 99.2%
Validation accuracy: 74.7%
Minibatch loss at step 700: 156.016739
Minibatch accuracy: 100.0%
Validation accuracy: 74.7%
Minibatch loss at step 800: 127.731316
Minibatch accuracy: 100.0%
Validation accuracy: 74.5%
Minibatch loss at step 900: 104.570061
Minibatch accuracy: 100.0%
Validation accuracy: 74.8%
Minibatch loss at step 1000: 85.609085
Minibatch accuracy: 100.

# Problem 4

In [66]:
batch_size = 128
nodes=1024
nodes_2=788
nodes_3=606
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  global_step = tf.Variable(0)
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal(
        [image_size * image_size, nodes],
        stddev=np.sqrt(2.0 / (image_size * image_size)))
    )
  biases_1 = tf.Variable(tf.zeros([nodes]))
  weights_2 = tf.Variable(
    tf.truncated_normal([nodes, nodes_2], stddev=np.sqrt(2.0 / nodes)))
  biases_2 = tf.Variable(tf.zeros([nodes_2]))
  weights_3 = tf.Variable(
    tf.truncated_normal([nodes_2, nodes_3], stddev=np.sqrt(2.0 / nodes_2)))
  biases_3 = tf.Variable(tf.zeros([nodes_3]))
  weights_4 = tf.Variable(
    tf.truncated_normal([nodes_3, num_labels], stddev=np.sqrt(2.0 / nodes_3)))
  biases_4 = tf.Variable(tf.zeros([num_labels]))
    
  # Training computation.
  layer_1_training=tf.nn.relu(tf.matmul(tf_train_dataset,weights_1)+biases_1)
  layer_2_training=tf.nn.relu(tf.matmul(layer_1_training,weights_2)+biases_2)
  layer_3_training=tf.nn.relu(tf.matmul(layer_2_training,weights_3)+biases_3)
  logits = tf.matmul(layer_3_training, weights_4) + biases_4
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 3500, 0.65, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  layer_1_validation=tf.nn.relu(tf.matmul(tf_valid_dataset,weights_1)+biases_1)
  layer_2_validation=tf.nn.relu(tf.matmul(layer_1_validation,weights_2)+biases_2)
  layer_3_validation=tf.nn.relu(tf.matmul(layer_2_validation,weights_3)+biases_3)
  layer_1_test=tf.nn.relu(tf.matmul(tf_test_dataset,weights_1)+biases_1)
  layer_2_test=tf.nn.relu(tf.matmul(layer_1_test,weights_2)+biases_2)
  layer_3_test=tf.nn.relu(tf.matmul(layer_2_test,weights_3)+biases_3)
  valid_prediction = tf.nn.softmax(
    tf.matmul(layer_3_validation, weights_4) + biases_4)
  test_prediction = tf.nn.softmax(tf.matmul(layer_3_test, weights_4) + biases_4)

In [69]:
num_steps = 48001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.342618
Minibatch accuracy: 13.3%
Validation accuracy: 16.1%
Minibatch loss at step 500: 0.359547
Minibatch accuracy: 89.8%
Validation accuracy: 86.0%
Minibatch loss at step 1000: 0.281639
Minibatch accuracy: 90.6%
Validation accuracy: 88.2%
Minibatch loss at step 1500: 0.342226
Minibatch accuracy: 89.8%
Validation accuracy: 88.2%
Minibatch loss at step 2000: 0.398256
Minibatch accuracy: 87.5%
Validation accuracy: 89.1%
Minibatch loss at step 2500: 0.168622
Minibatch accuracy: 94.5%
Validation accuracy: 89.2%
Minibatch loss at step 3000: 0.336882
Minibatch accuracy: 89.8%
Validation accuracy: 89.9%
Minibatch loss at step 3500: 0.265816
Minibatch accuracy: 89.8%
Validation accuracy: 90.2%
Minibatch loss at step 4000: 0.143458
Minibatch accuracy: 96.9%
Validation accuracy: 90.2%
Minibatch loss at step 4500: 0.226157
Minibatch accuracy: 91.4%
Validation accuracy: 90.3%
Minibatch loss at step 5000: 0.198691
Minibatch accuracy: 95.3%
Validation accurac